In [1]:
import requests,pprint,json,datetime,time
from pymongo import MongoClient
from secrets import *
# Sign up for free API key at https://openweathermap.org/appid

# 3.1 Acquring data from an API

In [2]:
city='london'
pprint(key)

NameError: name 'key' is not defined

In [4]:
requestString=u'https://api.openweathermap.org/data/2.5/weather?q=%s&APPID=%s' % (city,key)

NameError: name 'key' is not defined

In [4]:
res=requests.get(requestString)

In [5]:
res.reason

'OK'

In [6]:
pprint.pprint(res.json())

{u'base': u'stations',
 u'clouds': {u'all': 24},
 u'cod': 200,
 u'coord': {u'lat': 51.51, u'lon': -0.13},
 u'dt': 1511648400,
 u'id': 2643743,
 u'main': {u'humidity': 78,
           u'pressure': 1020,
           u'temp': 275.95,
           u'temp_max': 277.15,
           u'temp_min': 274.15},
 u'name': u'London',
 u'rain': {u'3h': 0.44},
 u'sys': {u'country': u'GB',
          u'id': 5091,
          u'message': 0.0037,
          u'sunrise': 1511595397,
          u'sunset': 1511625520,
          u'type': 1},
 u'visibility': 10000,
 u'weather': [{u'description': u'light rain',
               u'icon': u'10n',
               u'id': 500,
               u'main': u'Rain'}],
 u'wind': {u'deg': 230, u'speed': 5.1}}


In [20]:
def getData(city):
    requestString=u'https://api.openweathermap.org/data/2.5/weather?q=%s&APPID=%s' % (city,key)
    
    res=requests.get(requestString)
    
    return res

In [9]:
res=getData('La paz')

In [10]:
res

<Response [200]>

# 3.2 Ingesting data into MongoDB

## Create DB and collection

In [22]:
client=MongoClient('localhost')
db=client.packt

In [23]:
weatherCollection=db.weather

In [24]:
res=weatherCollection.insert_one(res.json())

## Get list of cities

In [14]:
!wget http://bulk.openweathermap.org/sample/city.list.json.gz

--2017-11-19 21:01:02--  http://bulk.openweathermap.org/sample/city.list.json.gz
Resolving bulk.openweathermap.org (bulk.openweathermap.org)... 78.46.48.103
Connecting to bulk.openweathermap.org (bulk.openweathermap.org)|78.46.48.103|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4292641 (4.1M) [application/octet-stream]
Saving to: ‘city.list.json.gz’

city.list.json.gz   100%[===================>]   4.09M  3.09MB/s    in 1.3s    

2017-11-19 21:01:03 (3.09 MB/s) - ‘city.list.json.gz’ saved [4292641/4292641]



In [15]:
!gunzip city.list.json.gz

In [16]:
!head city.list.json

[
  {
    "id": 707860,
    "name": "Hurzuf",
    "country": "UA",
    "coord": {
      "lon": 34.283333,
      "lat": 44.549999
    }
  },


In [12]:
with open('data/city.list.json','r') as inFile:
    citiesJson=json.loads(inFile.read())

In [13]:
citiesJson[0]

{u'coord': {u'lat': 44.549999, u'lon': 34.283333},
 u'country': u'UA',
 u'id': 707860,
 u'name': u'Hurzuf'}

In [14]:
type(citiesJson)

list

## Limit to Chilean cities

In [15]:
citiesJsonCL=filter(lambda x:x[u'country']==u'CL',citiesJson)

In [16]:
len(citiesJsonCL)

369

In [17]:
cities=map(lambda x:x['name'],citiesJsonCL)
ids=map(lambda x:x['id'],citiesJsonCL)

In [18]:
res=getData(cities[0])

In [19]:
res

<Response [200]>

## Cycle through cities

In [55]:
for i,name in zip(ids,cities):
    
    res=getData(name)
    
    if not res.status_code==200:
        print 'Error grabbing data for %s' % name
        print res.reason
        
    else:
        try:
            weatherCollection.insert_one(res.json())
        except e:
            print 'Error inserting into DB' % e
            print '(City %s)' % name
    
    time.sleep(1)
    # Sleep so we dont thrash the API

Error grabbing data for Villa Presidente Frei, Nunoa, Santiago, Chile
Not Found


In [219]:
def getTimestamp(dt):
    return pd.datetime.fromtimestamp(dt)

In [212]:
def getDate(tstamp):
    dt=datetime.datetime.fromtimestamp(tstamp)
    return dt.strftime('%b %d - %H:%m')

# 3.3 Querying MongoDB for useful information

In [228]:
cur=weatherCollection.find()

In [229]:
sortedCur=cur.sort('main.temp')

In [230]:
for doc in sortedCur.limit(10):
    try:
        print doc['name']
        print doc['main']['temp']
    except:
        print 'Error: missing name/temp'

Error: missing name/temp
Error: missing name/temp
Error: missing name/temp
Error: missing name/temp
Cochrane
264.09
Cochrane
266.63
Cañete
267.87
Los Álamos
267.946
Cochrane
268.67
Cañete
268.696


In [31]:
cur.count()

3046

## Find the max recorded temperatures over time per station

In [79]:
pipeline=[]
pipeline.append({'$group':{'_id':'$name','maxTemp':{'$max':'$main.temp'}}})
# Group measurements by city name, extract maximum recorded tmep for each
pipeline.append({'$limit':10})
# Limit results to first 10 cities

In [80]:
cur=weatherCollection.aggregate(pipeline=pipeline)

In [81]:
for d in cur:
    print d['_id'],d['maxTemp']

La Paz 293.15
None None
Provincia de Santiago 304.09
Las Gaviotas 297.15
Putre 287.486
Arica 295.24
Santiago 304.15
Futaleufu 288.236
El Arrayán 300.15
Puerto Bories 292.15


## Get datetime of max temp per station

In [192]:
pipeline=[]
pipeline.append({'$match':{'name':{'$exists':True}}})
# Filter out dirty rows
pipeline.append({'$sort':{'name':1,'main.temp':-1}})
# Sort by name (esc) and temperature (asc)
pipeline.append({'$group':{'_id':'$name','maxTemp':{'$first':'$main.temp'},'date':{'$first':'$dt'}}})
# Group by name, grab maximum temperature and date of temperature
# Sorted by temp so grab first document from each group
pipeline.append({'$limit':10})
# limit to first 10 cities

In [216]:
cur=weatherCollection.aggregate(pipeline=pipeline)

In [217]:
for d in cur:
    print getDate(d['date']),d['_id'],d['maxTemp']
    print ''

Nov 25 - 19:11 Región Aisén del General Carlos Ibáñez del Campo 277.736

Nov 25 - 19:11 Región de Magallanes y de la Antártica Chilena 280.886

Nov 25 - 19:11 Caletones 275.711

Nov 21 - 21:11 Puerto Williams 281.15

Nov 21 - 21:11 Lebu 286.308

Nov 25 - 19:11 Putre 287.486

Nov 25 - 19:11 Puerto Quellon 287.611

Nov 20 - 13:11 Cañete 287.836

Nov 25 - 19:11 Futaleufu 288.236

Nov 25 - 19:11 Coihaique 288.59



## Do sanity check for one city

In [214]:
for d in weatherCollection.find({'name':'Caletones'}):
    print getDate(d['dt']),'',d['main']['temp']

Nov 19 - 21:11  274.693
Nov 19 - 23:11  273.395
Nov 20 - 13:11  273.886
Nov 20 - 22:11  273.346
Nov 21 - 21:11  274.608
Nov 22 - 23:11  273.721
Nov 25 - 00:11  273.242
Nov 25 - 19:11  275.711
